In [1]:
from google.colab import drive
drive.mount('/content/drive/')
path = '/content/drive/MyDrive/SC4000/'

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb

xTrain=pd.read_csv(path+'cleaned_train_2.csv')
xTest=pd.read_csv(path+'cleaned_test_2.csv')

pd.set_option('display.max_columns', 300)
pd.set_option('float_format', '{:f}'.format)

Mounted at /content/drive/


Data Cleaning and Preprocessing: Not my work

In [2]:
yTrain = xTrain['price_doc'].apply(lambda x: np.log1p(x))
xTrain.drop(columns=['id','timestamp','price_doc'],axis=1,inplace=True)

print(xTrain.shape)

(30471, 279)


In [3]:
#Split train data into 2 parts
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(xTrain, yTrain, test_size=0.15,random_state=42)

In [4]:
print(X_tr.isnull().values.any())
print(X_val.isnull().values.any())

False
False


In [5]:
from sklearn import preprocessing

numerical_cols = xTrain.select_dtypes(exclude=['object'])
categorical_cols = xTrain.select_dtypes(include=['object']).copy()

# Encoding for categorical columns:
for c in categorical_cols:
  le = preprocessing.LabelEncoder()
  le.fit(X_tr[c])

  X_val[c] = X_val[c].map(lambda s: '<unknown>' if s not in le.classes_ else s)
  le.classes_ = np.append(le.classes_, '<unknown>')

  X_tr[c] = le.transform(X_tr[c])
  X_val[c] = le.transform(X_val[c])

# Scaling for numercial columns:
for c in numerical_cols:
  min = X_tr[c].min()
  max = X_tr[c].max()

  X_tr[c] = (X_tr[c] - min)/(max-min)
  X_val[c] = (X_val[c] - min)/(max-min)

In [6]:
print(X_tr.isnull().values.any())
print(X_val.isnull().values.any())

False
False


# **My work begins here:**

**Ridge Regressor**: Good for when the features are highly interlinked/correlated amongst themselves

In [7]:
from sklearn.linear_model import Ridge

# Initialize Ridge Regression
ridge = Ridge(alpha=0.0002)

In [8]:
# Fit the model
ridge.fit(X_tr, y_tr)

Ridge(alpha=0.0002)

In [9]:
from sklearn.metrics import mean_squared_error

y_pred_val = ridge.predict(X_val)
rmse_ridge = np.sqrt(mean_squared_error(y_val, y_pred_val))
print(f"Ridge RMSE on validation set: {rmse_ridge}")

Ridge RMSE on validation set: 0.7797888789732341


**Huber Regressor**: Good when there are lots of outliers

In [11]:
# from sklearn.linear_model import HuberRegressor

# Initialize HuberRegressor
# huber = HuberRegressor(max_iter=15000)

# Fit the model
# huber.fit(X_tr, y_tr)

HuberRegressor(max_iter=15000)

In [12]:
# y_pred_val_huber = huber.predict(X_val)
# rmse_huber = np.sqrt(mean_squared_error(y_val, y_pred_val_huber))
# print(f"Huber RMSE on validation set: {rmse_huber}")

Huber RMSE on validation set: 1.8567407675500478


In [10]:
## check for overfitting:

# For Ridge Regressor:
y_pred_train_ridge = ridge.predict(X_tr)
rmse_train_ridge = np.sqrt(mean_squared_error(y_tr, y_pred_train_ridge))
print(f"Ridge RMSE on training set: {rmse_train_ridge}")

Ridge RMSE on training set: 0.4880393715071023


In [ ]:
# For Huber Regressor:
# y_pred_train_huber = huber.predict(X_tr)
# rmse_train_huber = np.sqrt(mean_squared_error(y_tr, y_pred_train_huber))
# print(f"Huber RMSE on training set: {rmse_train_huber}")

Ridge Regression seems to be working well without overfitting. Huber Regressor is overfitting.

**Random Forest:**

In [13]:
from sklearn.ensemble import RandomForestRegressor

# Create a Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_tr, y_tr)

# Predictions
y_pred = rf.predict(X_val)

# Evaluate the model
mse = mean_squared_error(y_val, y_pred)

print("Mean Squared Error:", mse)


Mean Squared Error: 0.2391617696672002


In [14]:
print("Random forest RMSE =", np.sqrt(mse))

Random forest RMSE = 0.48904168499955114


# **Hyperparameter Tuning:**

**Ridge Regressor:**

In [90]:
from sklearn.linear_model import RidgeCV

# Define a range of alpha values to test
alphas = np.logspace(-4, 4, 100)

# Initialize RidgeCV with the alpha values and cross-validation strategy
ridge_cv = RidgeCV(alphas=alphas, store_cv_values=True)

# Fit the model
ridge_cv.fit(X_tr, y_tr)

# The best alpha value found
best_alpha = ridge_cv.alpha_
print(f"The best value of alpha is: {best_alpha}")


The best value of alpha is: 2.310129700083163


In [91]:
ridge = Ridge(alpha=best_alpha)

In [92]:
# Fit the model
ridge.fit(X_tr, y_tr)

Ridge(alpha=2.310129700083163)

In [93]:
from sklearn.metrics import mean_squared_error

y_pred_val = ridge.predict(X_val)
rmse_ridge = np.sqrt(mean_squared_error(y_val, y_pred_val))
print(f"Ridge RMSE on validation set: {rmse_ridge}")

Ridge RMSE on validation set: 0.5837074607691141


Better RMSE value = 0.58

In [30]:
## check for overfitting:

y_pred_train_ridge = ridge.predict(X_tr)
rmse_train_ridge = np.sqrt(mean_squared_error(y_tr, y_pred_train_ridge))
print(f"Ridge RMSE on training set: {rmse_train_ridge}")

Ridge RMSE on training set: 0.4969026056894369


Generate Ridge predictions CSV file:

*but first clean test data*

In [17]:
xTest = pd.read_csv(path+'cleaned_test_2.csv')

In [18]:
testID = xTest['id']
xTest.drop(['id', 'timestamp'],axis=1,inplace=True)

print(xTest.shape)


(7662, 279)


In [19]:
num = xTrain.select_dtypes(exclude=['object'])
cat = xTrain.select_dtypes(include=['object']).copy()


for c in cat:
  le = preprocessing.LabelEncoder()
  le.fit(xTrain[c])

  xTest[c] = xTest[c].map(lambda s: '<unknown>' if s not in le.classes_ else s)
  le.classes_ = np.append(le.classes_, '<unknown>')

  xTest[c] = le.transform(xTest[c])

for c in num:
  min = xTrain[c].min()
  max = xTrain[c].max()

  xTest[c] = (xTest[c] - min)/(max-min)

Predict using ridge on test data:

In [20]:
xTest.head()

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,radiation_raion,big_market_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,railroad_station_avto_km,railroad_station_avto_min,public_transport_station_km,public_transport_station_min_walk,water_km,water_1line,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,big_road2_km,railroad_km,zd_vokzaly_avto_km,bus_terminal_avto_km,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,trc_count_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,big_church_count_500,church_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,trc_count_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,big_church_count_1000,church_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,big_church_count_1500,church_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_avg,cafe_avg_price_2000,cafe_count_2000_na_price,cafe_count_2000_price_500,cafe_count_2000_price_1000,cafe_count_2000_price_1500,cafe_count_2000_price_2500,cafe_count_2000_price_4000,big_church_count_2000,church_count_2000,mosque_count_2000,leisure_count_2000,sport_count_2000,market_count_2000,green_part_3000,prom_part_3000,office_count_3000,office_sqm_3000,trc_count_3000,trc_sqm_3000,cafe_count_3000,cafe_sum_3000_min_price_avg,cafe_sum_3000_max_price_avg,cafe_avg_price_3000,cafe_count_3000_na_price,cafe_cou

In [100]:
predictions = ridge.predict(xTest)

In [106]:
submission_df = pd.DataFrame({
    'Id': testID,
    'price_doc': np.expm1(predictions)
})
submission_df.head(10)

,Id,price_doc
0,30474,2681097.282146
1,30475,6126743.317951
2,30476,3805095.709179
3,30477,5517163.450757
4,30478,4222991.288100
5,30479,4889415.601213
6,30480,4487364.040485
7,30481,3405759.366537
8,30482,5137010.126569
9,30483,3902465.476387


In [107]:
submission_file_path = '/mnt/data/submission.csv'
submission_df.to_csv('output_file2.csv',index=False)

In [108]:
submission_file_path

'/mnt/data/submission.csv'

###################################################################################

**Huber Regressor:**

In [37]:
from sklearn.model_selection import RandomizedSearchCV

# Define a distribution of epsilon values to test
epsilon_values = np.linspace(1.35, 1.5, 10)

# Set up the HuberRegressor with a smaller number of iterations for faster search
huber = HuberRegressor(max_iter=1000)

# Create a dictionary with the parameters to test
param_distributions = {'epsilon': epsilon_values}

# Initialize RandomizedSearchCV with the HuberRegressor and the distribution of epsilon values
# n_iter controls the number of different combinations to try
# n_jobs=-1 will use all available cores for parallel processing
random_search = RandomizedSearchCV(huber, param_distributions, n_iter=5, cv=3,
                                   scoring='neg_mean_squared_error', n_jobs=-1, random_state=42)

# Fit the random search to the data
random_search.fit(X_tr, y_tr)

# Find the best epsilon value and the corresponding best HuberRegressor
best_epsilon = random_search.best_params_['epsilon']
best_huber_regressor = random_search.best_estimator_

print(f"The best value of epsilon is: {best_epsilon}")




The best value of epsilon is: 1.4833333333333334


Best epsilon = 1.483

In [39]:
best_huber_regressor = HuberRegressor(epsilon=best_epsilon, max_iter=15000)
best_huber_regressor.fit(X_tr, y_tr)


HuberRegressor(epsilon=1.4833333333333334, max_iter=15000)

In [40]:
y_pred_val = best_huber_regressor.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
print(f"RMSE on validation set: {rmse}")


RMSE on validation set: 1.924523227034524


RMSE After hyperparameter tuning: 1.92

In [41]:
# check for overfitting:
y_pred_train = best_huber_regressor.predict(X_tr)
rmse_train = np.sqrt(mean_squared_error(y_tr, y_pred_train))
print(f"Huber RMSE on training set: {rmse_train}")

Huber RMSE on training set: 0.9826997231286873


On training set, RMSE = 0.98

Overall, Ridge performed much better than Huber.

###################################################################################

**Random Forest:**

In [15]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error

# Define a distribution of hyperparameters to test
param_distributions = {
    'n_estimators': [100, 150],  # More estimators may yield better results but use more memory
    'max_depth': [10, 20, None],  # Control the depth of the trees
    'min_samples_split': [2, 5],  # Minimum number of samples required to split a node
    'min_samples_leaf': [1, 2],   # Minimum number of samples required at a leaf node
    'max_features': ['auto', 'sqrt']  # Number of features to consider at each split
}

# Initialize the Random Forest Regressor
rf = RandomForestRegressor(random_state=42)

# Initialize RandomizedSearchCV with the random forest and the distribution of hyperparameters
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_distributions,
    n_iter=10,  # Number of parameter settings that are sampled
    cv=3,  # 3-fold cross-validation
    scoring='neg_mean_squared_error',  # Use MSE to evaluate the models
    n_jobs=-1,  # Use all available cores
    random_state=42
)

# Fit the random search to the data
random_search.fit(X_tr, y_tr)

# Print the best parameters found
print("Best parameters found: ", random_search.best_params_)

# Evaluate the best model on the validation set
best_model = random_search.best_estimator_
y_pred_val = best_model.predict(X_val)
mse = mean_squared_error(y_val, y_pred_val)
rmse = np.sqrt(mse)
print(f"Random Forest RMSE on validation set: {rmse}")


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Best parameters found:  {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 10}
Random Forest RMSE on validation set: 0.4708791676241466


In [21]:
predictions = best_model.predict(xTest)

In [22]:
submission_df = pd.DataFrame({
    'Id': testID,
    'price_doc': np.expm1(predictions)
})
submission_df.head(10)

,Id,price_doc
0,30474,5024551.060117
1,30475,8266149.567153
2,30476,5254128.301050
3,30477,5710166.191295
4,30478,4985885.883837
5,30479,7479282.240024
6,30480,4286023.806801
7,30481,4260851.829717
8,30482,3971718.683137
9,30483,4431383.174053


In [24]:
submission_file_path = '/mnt/data/submission.csv'
submission_df.to_csv('output_file3.csv',index=False)

In [26]:
# check overfitting or not
y_pred_train_rf = best_model.predict(X_tr)
rmse_train_rf = np.sqrt(mean_squared_error(y_tr, y_pred_train_rf))
print(f"RF RMSE on training set: {rmse_train_rf}")

RF RMSE on training set: 0.3896953138216551
